In [115]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets as dts
import math
import sys

*Создаем класс Баеса*

In [116]:
class Gaussian_Naive_Baes():

    def __init__(self):
        pass

    def fit(self, X, y):

        self.y_unique = np.unique(y)

        self.values = np.array([X[yv == y] for yv in self.y_unique]) # создаем массивы для классов
        # print(self.values[0].mean(axis=0))

        # Находим среднеквадратичную ошибку и среднее значения атрбутов
        self.mean = [xv.mean(axis=0) for xv in self.values]
        self.stdev = [xv.std(axis=0) for xv in self.values]
        # print(self.mean)

        pass

    # Классификатор массива
    def predict(self, X):
        return [self.predict_proba(x) for x in X]

    # Классификатор
    def predict_proba(self, X):

        predict_probabilities = {k:1 for k in self.y_unique}

        # Перемножаем вероятности атрибутов
        for yv in self.y_unique:
            for feature in range(len(self.mean)):
                predict_probabilities[yv] *= self.calculate_probability(X[feature], self.mean[yv][feature], self.stdev[yv][feature])

        return max(predict_probabilities, key=predict_probabilities.get)

    def calculate_probability(self, x, mean, stdev):
        # Вычисляем вероятность по формуле
        exponent = np.exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
        return (1 / (np.sqrt(2 * np.pi) * stdev)) * exponent

    def score(self, X, y):
        rez = np.array(self.predict(X))
        return rez[y == rez].size / rez.size


In [117]:
def score_categorical_bayes(X, y):

    clf = GaussianNB()

    # obj_df[obj_df.isnull().any(axis=1)] # Очистка нулевых значений

    # KFold
    from sklearn.model_selection import KFold

    # инициализация KFold с 5тью фолдами
    kf = KFold(n_splits=5)
    classifiers = []

    # метод split возвращает индексы для объектов train и test
    for train_index, test_index in kf.split(X):
        clf_cross = GaussianNB()
        X_train, X_test = X[train_index[0]:train_index[-1]], X[test_index[0]:test_index[-1]]
        y_train, y_test = y[train_index[0]:train_index[-1]], y[test_index[0]:test_index[-1]]
        clf_cross.fit(X_train, y_train)
        classifiers.append(clf_cross.score(X_test, y_test)) # получим 5 оптимальных классификаторов
    print("KFold: ", classifiers) # Выводим рез-тат кроссвалидации

    # Делим данные на тестовые и тренировочные
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 42)

    clf.fit(x_train, y_train)
    print("GaussianNB() score: ", clf.score(x_test, y_test))

In [118]:
def score_my_naive_bayes(X, y):

    clf = Gaussian_Naive_Baes()

    # KFold
    from sklearn.model_selection import KFold

    # инициализация KFold с 5тью фолдами
    kf = KFold(n_splits=5)
    classifiers = []

    # метод split возвращает индексы для объектов train и test
    for train_index, test_index in kf.split(X):
        clf_cross = Gaussian_Naive_Baes()
        X_train, X_test = X[train_index[0]:train_index[-1]], X[test_index[0]:test_index[-1]]
        y_train, y_test = y[train_index[0]:train_index[-1]], y[test_index[0]:test_index[-1]]
        clf_cross.fit(X_train, y_train)
        classifiers.append(clf_cross.score(X_test, y_test)) # получим 5 оптимальных классификаторов
    print("KFold: ", classifiers) # Выводим рез-тат кроссвалидации

    # Делим данные на тестовые и тренировочные
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 42)

    from sklearn import preprocessing

    min_max_scaler = preprocessing.MinMaxScaler() # Создаем объект для нормализации
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)


    clf.fit(x_train, y_train)
    print("Gaussian_Naive_Baes score: ", clf.score(x_test, y_test) )

In [119]:
def main():
    # Загружаем датасеты
    for load_dts, name_dts in [(dts.load_iris, "Iris plants dataset"),
                               (dts.load_digits, "Optical recognition of handwritten digits dataset"),
                               (dts.load_wine,"Wine recognition dataset"),
                               (dts.load_breast_cancer, "Breast cancer wisconsin (diagnostic) dataset")]:


        data = load_dts(); # Загружаем датасеты

        # Разделяем данные
        X = data.data
        y = data.target

        print("<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ ДЛЯ", name_dts, "--------------------------------->\n")

        print("Sklearn GaussianNB:")
        score_categorical_bayes(X, y)
        print()

        print("My work GaussianNB:")
        score_my_naive_bayes(X, y)
        print()

In [120]:
if __name__ == "__main__":
    main()

<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ ДЛЯ Iris plants dataset --------------------------------->

Sklearn GaussianNB:
KFold:  [1.0, 0.9655172413793104, 0.9310344827586207, 0.9655172413793104, 0.8620689655172413]
GaussianNB() score:  0.9777777777777777

My work GaussianNB:
KFold:  [1.0, 0.9310344827586207, 0.8620689655172413, 0.8620689655172413, 0.7586206896551724]
[[ 1.62924054  1.25276297  0.33647224 -1.60943791]
 [ 1.58923521  1.09861229  0.33647224 -1.60943791]
 [ 1.54756251  1.16315081  0.26236426 -1.60943791]
 [ 1.5260563   1.13140211  0.40546511 -1.60943791]
 [ 1.60943791  1.28093385  0.33647224 -1.60943791]
 [ 1.68639895  1.36097655  0.53062825 -0.91629073]
 [ 1.5260563   1.22377543  0.33647224 -1.2039728 ]
 [ 1.60943791  1.22377543  0.40546511 -1.60943791]
 [ 1.48160454  1.06471074  0.33647224 -1.60943791]
 [ 1.58923521  1.13140211  0.40546511 -2.30258509]
 [ 1.68639895  1.30833282  0.40546511 -1.60943791]
 [ 1.56861592  1.22377543  0.47000363 -1.60943791]
 [ 1.56861592  1.098

C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.values = np.array([X[yv == y] for yv in self.y_unique]) # создаем массивы для классов
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.values = np.array([X[yv == y] for yv in self.y_unique]) # создаем массивы для классов


<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ ДЛЯ Optical recognition of handwritten digits dataset --------------------------------->

Sklearn GaussianNB:
KFold:  [0.7883008356545961, 0.8635097493036211, 0.8519553072625698, 0.9078212290502793, 0.8128491620111732]
GaussianNB() score:  0.8518518518518519

My work GaussianNB:


C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.values = np.array([X[yv == y] for yv in self.y_unique]) # создаем массивы для классов
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:38: RuntimeWarning: invalid value encountered in double_scalars
  exponent = np.exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:39: RuntimeWarning: divide by zero encountered in double_scalars
  return (1 / (np.sqrt(2 * np.pi) * stdev)) * exponent
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:38: RuntimeWarning: divide by zero encountered in double_scalars
  exponent = np.exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
C:\Users\kuzin

KFold:  [0.10584958217270195, 0.10027855153203342, 0.09217877094972067, 0.09776536312849161, 0.09776536312849161]
[[      -inf       -inf 1.60943791 ...       -inf       -inf       -inf]
 [      -inf       -inf       -inf ... 2.30258509       -inf       -inf]
 [      -inf       -inf       -inf ... 2.77258872 2.19722458       -inf]
 ...
 [      -inf       -inf 0.         ... 1.79175947       -inf       -inf]
 [      -inf       -inf 0.69314718 ... 2.48490665       -inf       -inf]
 [      -inf       -inf 2.30258509 ... 2.48490665 0.               -inf]]


C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/1127956313.py:21: RuntimeWarning: divide by zero encountered in log
  print(np.log(X))
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.values = np.array([X[yv == y] for yv in self.y_unique]) # создаем массивы для классов
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:38: RuntimeWarning: invalid value encountered in double_scalars
  exponent = np.exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:39: RuntimeWarning: divide by zero encountered in double_scalars
  return (1 / (np.sqrt(2 * np.pi) * stdev)) * exponent
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:3

Gaussian_Naive_Baes score:  0.09814814814814815

<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ ДЛЯ Wine recognition dataset --------------------------------->

Sklearn GaussianNB:
KFold:  [0.9428571428571428, 0.9714285714285714, 0.9714285714285714, 1.0, 0.9117647058823529]
GaussianNB() score:  1.0

My work GaussianNB:
KFold:  [0.8857142857142857, 0.7142857142857143, 0.8857142857142857, 0.7941176470588235, 0.6764705882352942]
[[ 2.65535241  0.53649337  0.88789126 ...  0.03922071  1.36609165
   6.97073008]
 [ 2.58021683  0.57661336  0.76080583 ...  0.04879016  1.22377543
   6.95654544]
 [ 2.57718193  0.85866162  0.98207847 ...  0.0295588   1.15373159
   7.07749805]
 ...
 [ 2.58550585  1.45395301  0.81536481 ... -0.52763274  0.44468582
   6.72743172]
 [ 2.57794152  0.95165788  0.86288996 ... -0.51082562  0.48242615
   6.73340189]
 [ 2.6483002   1.41098697  1.00795792 ... -0.49429632  0.47000363
   6.32793678]]
Gaussian_Naive_Baes score:  0.8148148148148148

<-----------------ОЦЕНКИ КЛАССИФИКАЦИИ

C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.values = np.array([X[yv == y] for yv in self.y_unique]) # создаем массивы для классов
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.values = np.array([X[yv == y] for yv in self.y_unique]) # создаем массивы для классов
C:\Users\kuzin\AppData\Local\Temp/ipykernel_14776/2336579754.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (w